# Summary

+ Baseline  
https://dacon.io/competitions/official/235805/codeshare/3620?page=1&dtype=recent  
이전 다른 이미지 대회에서 수상했던 코드를 참고하였습니다.  
좋은 코드 공유 감사합니다.

+ Data Augmentation   
데이터 불균형이 심해서 학습 이전에 Anomaly한 데이터만 다양한 방법으로 증강하여 새로운 데이터 셋 폴더에 저장하고,  
csv 파일에도 추가한 데이터에 대한 정보를 합쳐서 새로 만들어 주었습니다.  
그리고, 학습 과정에서도 pytorch transforms을 활용하여 데이터를 증강시켜주었습니다.

+ Training parameter
    + Lamb Optimizer
    + ReduceLROnplateau Scheduler
    + WarmUpLR Scheduler
    + Focal Loss
    + Early Stopping


+ Model  
모델은 5 k-fold로 진행하면서 Public 기준으로 결과가 좋은 상위 2개 모델을 골라서 앙상블을 진행해주었습니다.   
그리고, 2가지 모델에 대해서 network 구조와, transforms 방법을 조금 다르게 설정 하였습니다. 다른 모델은 주석으로 표시해두었습니다.
    + cait_s36_384, img_size: 384
    + tf_efficientnet_b7_ns, img_size: 600

+ TTA  
https://github.com/qubvel/ttach  
d4_transform()

# Library

In [ ]:
import os
import cv2
import time
import random
import logging
import easydict
import numpy as np
import pandas as pd
from tqdm import tqdm
from os.path import join as opj
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from PIL import Image
from natsort import natsorted

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms
from torch import Tensor
from torchvision.transforms import functional as F
import torch.cuda.amp as amp
from adamp import AdamP, SGDP

import warnings
warnings.filterwarnings('ignore')

# Config

In [ ]:
args = easydict.EasyDict(
    {'exp_num':'0',
     
     # Path settings
     'data_path':'./open',
     'Kfold':5,
     
     'model_path':'label_results_cait_s36_384/',
#      'model_path':'label_results_tf_efficientnet_b7_ns_600/',

     'image_type':'train_1024', 
     'class_num' : 88,

     # Model parameter settings
     'model_name':'cait_s36_384',
#      'model_name':'tf_efficientnet_b7_ns',
     
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':384,
#      'img_size':600, # tf_efficientnet_b7_ns
     
     'batch_size':16,
     'epochs':100,
     'optimizer':'Lamb',
     'initial_lr':5e-4,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,

     ## Scheduler (OnecycleLR)
     'scheduler':'Reduce',
     'warm_epoch':5,
     'max_lr':1e-3,

     ### Cosine Annealing
     'min_lr':5e-5,
     'tmax':145,

     ## etc.
     'patience': 5,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':True,
     'logging':False,
     'num_workers':4,
     'seed':42
     
     
    })

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPUs 0 and 1 to use

# Data Augmentation

train 폴더를 train_add_data 이름으로 새로운 폴더에 복사 후 진행

In [ ]:
DATA_DIR = './open'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df.csv'))
train_df = train_df.drop('index', axis=1)

In [ ]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

# Random_Crop

In [ ]:
def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]

    return crop

In [ ]:
oslabel = list(train_df['label'].unique())


for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = cv2.resize(image, dsize=(1024, 1024))
            aug_img = get_random_crop(aug_img, 900, 900)
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_crop.png')
            save_name = f'{label}_{idx}_crop.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()

# Rotation

In [ ]:
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h)) 
    return img

In [ ]:
for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = rotation(image, 30)  
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_rotation.png')
            save_name = f'{label}_{idx}_rotation.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist() 

# Flip

In [ ]:
for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = cv2.flip(image, 1)
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_flip1.png')
            save_name = f'{label}_{idx}_flip1.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()   

In [ ]:
for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = cv2.flip(image, 2)
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_flip2.png')
            save_name = f'{label}_{idx}_flip2.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()   

# label 인코딩 후 증강된 csv 저장

In [ ]:
train_labels = train_df["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]
train_labels

In [ ]:
train_df['encoder_label'] = train_labels

In [ ]:
train_df.to_csv('./open/train_df_add_data.csv')

# 증강된 데이터 셋 불러오기

In [ ]:
DATA_DIR = './open'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df_add_data.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test_df.csv'))

print(train_df.head())
print(test_df.head())
print(train_df.shape)
print(test_df.shape)

   Unnamed: 0  file_name       class state            label  encoder_label
0           0  10000.png  transistor  good  transistor-good             72
1           1  10001.png     capsule  good     capsule-good             15
2           2  10002.png  transistor  good  transistor-good             72
3           3  10003.png        wood  good        wood-good             76
4           4  10004.png      bottle  good      bottle-good              3
   index  file_name
0      0  20000.png
1      1  20001.png
2      2  20002.png
3      3  20003.png
4      4  20004.png
(13997, 6)
(2154, 2)


In [ ]:
train_df['label'].unique()

array(['transistor-good', 'capsule-good', 'wood-good', 'bottle-good',
       'screw-good', 'cable-bent_wire', 'carpet-hole', 'hazelnut-good',
       'pill-pill_type', 'cable-good', 'metal_nut-scratch', 'pill-good',
       'screw-thread_side', 'zipper-fabric_border', 'leather-good',
       'pill-scratch', 'toothbrush-good', 'hazelnut-crack',
       'screw-manipulated_front', 'zipper-good', 'tile-good',
       'carpet-good', 'metal_nut-good', 'bottle-contamination',
       'grid-good', 'zipper-split_teeth', 'pill-crack', 'wood-combined',
       'pill-color', 'screw-thread_top', 'cable-missing_cable',
       'capsule-squeeze', 'zipper-rough', 'capsule-crack', 'capsule-poke',
       'metal_nut-flip', 'carpet-metal_contamination', 'metal_nut-color',
       'transistor-bent_lead', 'zipper-fabric_interior', 'leather-fold',
       'tile-glue_strip', 'screw-scratch_neck', 'screw-scratch_head',
       'hazelnut-cut', 'bottle-broken_large', 'bottle-broken_small',
       'leather-cut', 'cable-cut_

In [ ]:
class_num = len(train_df.encoder_label.unique())
class_num

88

# Utils for training and Logging

In [ ]:
# Warmup Learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

# Logging
def get_root_logger(logger_name='basicsr',
                    log_level=logging.INFO,
                    log_file=None):

    logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
    if logger.hasHandlers():
        return logger

    format_str = '%(asctime)s %(levelname)s: %(message)s'
    logging.basicConfig(format=format_str, level=log_level)

    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    return logger

class AvgMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

# Dataset & Loader

In [ ]:
class Train_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.img_path = df['file_name'].values
        self.target = df['encoder_label'].values 
        self.transform = transform

        print(f'Dataset size:{len(self.img_path)}')

    def __getitem__(self, idx):
        
        image = Image.open(opj('./open/train_add_data/', self.img_path[idx])).convert('RGB')
        image = self.transform(image)
        target = self.target[idx]

        return image, target

    def __len__(self):
        return len(self.img_path)

class Test_dataset(Dataset):
    def __init__(self, df, transform=None):
        self.img_path = df['file_name'].values
        self.transform = transform

        print(f'Test Dataset size:{len(self.img_path)}')

    def __getitem__(self, idx):

        image = Image.open(opj('./open/test/', self.img_path[idx])).convert('RGB')
        image = self.transform(image)

        return image

    def __len__(self):
        return len(self.img_path)

def get_loader(df, phase: str, batch_size, shuffle,
               num_workers, transform):
    if phase == 'test':
        dataset = Test_dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    else:
        dataset = Train_Dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                 pin_memory=True,
                                 drop_last=False)
    return data_loader



# Dataset Augmentation

In [ ]:
class RandomRotation(transforms.RandomRotation):
    def __init__(self, p: float, degrees: int):
        super(RandomRotation, self).__init__(degrees)
        self.p = p

    def forward(self, img):
        if torch.rand(1) < self.p:
            fill = self.fill
            if isinstance(img, Tensor):
                if isinstance(fill, (int, float)):
                    fill = [float(fill)] * F.get_image_num_channels(img)
                else:
                    fill = [float(f) for f in fill]
            angle = self.get_params(self.degrees)

            img = F.rotate(img, angle, self.resample, self.expand, self.center, fill)
        return img

## cait_s36_384

In [ ]:
def get_train_augmentation(img_size, ver):
    if ver==1: # for validset
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])

    if ver == 2:

        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                transforms.RandomAffine((-20,20)),
                RandomRotation(0.5, degrees=5),
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(), 
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    
    
    return transform

## tf_efficientnet_b7_ns

In [ ]:
# def get_train_augmentation(img_size, ver):
#     if ver==1: # for validset
#         transform = transforms.Compose([
#                 transforms.Resize((img_size, img_size)),
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]),
#             ])

#     if ver == 2:

#         transform = transforms.Compose([
#                 transforms.RandomHorizontalFlip(),
#                 transforms.RandomVerticalFlip(),
#                 transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
#                 transforms.RandomAffine((-20,20)),
#                 RandomRotation(0.7, degrees=5),
#                 transforms.Resize((img_size, img_size)),
#                 transforms.ToTensor(), 
#                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]),
#             ])
    
    
#     return transform

# Network

## cait_s36_384

In [ ]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.model_ft = timm.create_model(args.model_name, pretrained=True, num_classes=88)

    def forward(self, x):
        out = self.model_ft(x)
        return out

class Network_test(nn.Module):
    def __init__(self, encoder_name):
        super().__init__()
        self.model_ft = timm.create_model(args.model_name, pretrained=True, num_classes=88)

    def forward(self, x):
        out = self.model_ft(x)
        return out

## tf_efficientnet_b7_ns

In [ ]:
# class Network(nn.Module):
#     def __init__(self, args):
#         super().__init__()
#         self.model_ft = timm.create_model(args.model_name, pretrained=True,  drop_path_rate=args.drop_path_rate,
#                                           num_classes=88)

        
#     def forward(self, x):
#         out = self.model_ft(x)
#         return out

# class Network_test(nn.Module):
#     def __init__(self, encoder_name):
#         super().__init__()
#         self.model_ft = timm.create_model(args.model_name, pretrained=True, num_classes=88)

#     def forward(self, x):
#         out = self.model_ft(x)
#         return out

# FocalLoss

In [ ]:
class FocalLoss(nn.Module):
    """
    https://dacon.io/competitions/official/235585/codeshare/1796
    """

    def __init__(self, gamma=2.0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        # print(self.gamma)
        self.eps = eps
        self.ce = nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

# Trainer for Training & Validation

In [ ]:
class Trainer():
    def __init__(self, args, save_path):
        '''
        args: arguments
        save_path: Model 가중치 저장 경로
        '''
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Logging
        log_file = os.path.join(save_path, 'log.log')
        self.logger = get_root_logger(logger_name='IR', log_level=logging.INFO, log_file=log_file)
        self.logger.info(args)
        # self.logger.info(args.tag)

        # Train, Valid Set load
        ############################################################################
        if args.step == 0 :
            df_train = pd.read_csv(opj(args.data_path, 'train_df_add_data.csv'))
        else :
            df_train = pd.read_csv(opj(args.data_path, f'train_{args.step}step.csv'))

#         if args.image_type is not None:
#             df_train['img_path'] = df_train['img_path'].apply(lambda x:x.replace('train_imgs', args.image_type))
#             df_train['img_path'] = df_train['img_path'].apply(lambda x:x.replace('test_imgs', 'test_1024'))

        kf = StratifiedKFold(n_splits=args.Kfold, shuffle=True, random_state=args.seed)
        for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(df_train)), y=df_train['encoder_label'])):
            df_train.loc[val_idx, 'fold'] = fold
        val_idx = list(df_train[df_train['fold'] == int(args.fold)].index)

        df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
        df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)

        # Augmentation
        self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)

        # TrainLoader
        self.train_loader = get_loader(df_train, phase='train', batch_size=args.batch_size, shuffle=True,
                                       num_workers=args.num_workers, transform=self.train_transform)
        self.val_loader = get_loader(df_val, phase='train', batch_size=args.batch_size, shuffle=False,
                                       num_workers=args.num_workers, transform=self.test_transform)

        # Network
        self.model = Network(args).to(self.device)

        # Loss
#         self.criterion = nn.CrossEntropyLoss()
        self.criterion = FocalLoss()
        
        # Optimizer & Scheduler
        self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr)
        
        iter_per_epoch = len(self.train_loader)
        self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

        if args.scheduler == 'step':
            self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=args.milestone, gamma=args.lr_factor, verbose=True)
        elif args.scheduler == 'cos':
            tmax = args.tmax # half-cycle 
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
        elif args.scheduler == 'cycle':
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)
        else:
            self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            
        if args.multi_gpu:
            self.model = nn.DataParallel(self.model).to(self.device)

        # Train / Validate
        best_loss = np.inf
        best_acc = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        for epoch in range(1, args.epochs+1):
            self.epoch = epoch

            if args.scheduler == 'cos':
                if epoch > args.warm_epoch:
                    self.scheduler.step()

            # Training
            train_loss, train_acc, train_f1 = self.training(args)

            # Model weight in Multi_GPU or Single GPU
            state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

            # Validation
            val_loss, val_acc, val_f1 = self.validate(args, phase='val')

            # Save models
            if val_loss < best_loss:
                early_stopping = 0
                best_epoch = epoch
                best_loss = val_loss
                best_acc = val_acc
                best_f1 = val_f1

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path, 'best_model.pth'))
                self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == args.patience:
                break

        self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f} | Val F1:{best_f1:.4f}')
        end = time.time()
        self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')

    # Training
    def training(self, args):
        self.model.train()
        train_loss = AvgMeter()
        train_acc = 0
        preds_list = []
        targets_list = []

        scaler = grad_scaler.GradScaler()
        
        for i, (images, targets) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            targets = torch.tensor(targets, device=self.device, dtype=torch.long)
            
            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)
    
            if args.amp:
                with autocast():
                    preds = self.model(images)
                    loss = self.criterion(preds, targets)
                    
                scaler.scale(loss).backward()

                # Gradient Clipping
                if args.clipping is not None:
                    scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

                scaler.step(self.optimizer)
                scaler.update()

            else:
                preds = self.model(images)
                loss = self.criterion(preds, targets)
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
                self.optimizer.step()

            if args.scheduler == 'cycle':
                if self.epoch > args.warm_epoch:
                    self.scheduler.step()

            # Metric
            train_acc += (preds.argmax(dim=1) == targets).sum().item()
            preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
            targets_list.extend(targets.cpu().detach().numpy())
            # log
            train_loss.update(loss.item(), n=images.size(0))

        train_acc /= len(self.train_loader.dataset)
        train_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

        self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
        self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f} | F1:{train_f1:.4f}')
        return train_loss.avg, train_acc, train_f1
            
    # Validation or Dev
    def validate(self, args, phase='val'):
        self.model.eval()
        with torch.no_grad():
            val_loss = AvgMeter()
            val_acc = 0
            preds_list = []
            targets_list = []

            for i, (images, targets) in enumerate(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                targets = torch.tensor(targets, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, targets)

                # Metric
                val_acc += (preds.argmax(dim=1) == targets).sum().item()
                preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
                targets_list.extend(targets.cpu().detach().numpy())

                # log
                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)
            val_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

            self.logger.info(f'{phase} Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f} | F1:{val_f1:.4f}')
        return val_loss.avg, val_acc, val_f1

# Main

In [ ]:
def main(args):
    print('<---- Training Params ---->')
    
    # Random Seed
    seed = args.seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

    save_path = os.path.join(args.model_path, (args.exp_num).zfill(3))
    
    # Create model directory
    os.makedirs(save_path, exist_ok=True)
    Trainer(args, save_path)

    return save_path

# TTA Inference & Ensemble_5fold

In [ ]:
import ttach as tta

def predict(encoder_name, test_loader, device, model_path):
    
    model = Network_test(encoder_name).to(device)
    model.load_state_dict(torch.load(opj(model_path, 'best_model.pth'))['state_dict'])
    
#     model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.d4_transform())
    tta_model.eval()
    
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = tta_model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

def ensemble_5fold(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:
        prediction = predict(encoder_name=args.model_name, test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)

    return ensemble


# Train

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 2


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sub = pd.read_csv('./open/sample_submission.csv')
df_train = pd.read_csv('./open/train_df_add_data.csv')
df_test = pd.read_csv('./open/test_df.csv')

In [ ]:
args.step = 0
models_path = []
for s_fold in range(5): # 5fold
    args.fold = s_fold
    args.exp_num = str(s_fold)
    save_path = main(args)
    models_path.append(save_path)

# Inference & Ensemble_Model

In [ ]:
img_size = 600
args.model_name = 'tf_efficientnet_b7_ns'

test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(df_test, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

Test Dataset size:2154


In [ ]:
models_path = ['./label_results_tf_efficientnet_b7_ns_600/000', 
               './label_results_tf_efficientnet_b7_ns_600/001', 
               './label_results_tf_efficientnet_b7_ns_600/002', 
               './label_results_tf_efficientnet_b7_ns_600/003', 
               './label_results_tf_efficientnet_b7_ns_600/004']

In [ ]:
ensemble1 = ensemble_5fold(models_path, test_loader, device)

In [ ]:
img_size = 384
args.model_name = 'cait_s36_384'

test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(df_test, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

Test Dataset size:2154


In [ ]:
models_path = ['./label_results_cait_s36_384/000', 
               './label_results_cait_s36_384/001', 
               './label_results_cait_s36_384/002', 
               './label_results_cait_s36_384/003', 
               './label_results_cait_s36_384/004']

In [ ]:
ensemble2 = ensemble_5fold(models_path, test_loader, device)

In [ ]:
ensemble = (ensemble1+ensemble2) / 2

In [ ]:
f_pred = ensemble.argmax(axis=1).tolist()
f_pred

# Submit

In [ ]:
train_y = pd.read_csv("./open/train_df_add_data.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}


In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
f_result

In [ ]:
submission = pd.read_csv("./open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv("./submission/final.csv", index = False)